I completed this problems as part of Udacity's technical interview practice. I'll admit I'm MUCH more of a statistician and machine learning engineer than a computer scientist, and I probably wouldn't be able to solve these problems on the spot. But given enough time, I'm clever and resourceful enough to figure these sorts of things out. :)

# Question 1

Given two strings s and t, determine whether some anagram of t is a substring of s. For example: if s = "udacity" and t = "ad", then the function returns True. Your function definition should look like: question1(s, t) and return a boolean True or False.

In [288]:
from itertools import permutations

def question1(s, t):
    if (type(s) != str) or (type(t) != str) or (len(s) == 0) or (len(t) == 0):
        return False
    if len(t) > 8:
        print("Error! It's going to take too long to process this. Try a shorter string for t.")
        return False
    anagrams = ["".join(i) for i in permutations(t, len(t))]
    for ana in anagrams:
        if ana in s:
            return True
    return False

In [289]:
print(question1('udacity', 'caity'))

True


In [291]:
print(question1('udacityudacityudacityudacity', 'caitycaitycaitycaity'))

Error! It's going to take too long to process this. Try a shorter string for t.
False


In [292]:
print(question1('udacity', ''))

False


I can think of at least one other way to solve this, but after discovering itertools a while back, I use it whenever I have a good excuse to. :) It's an efficient and compact way to create permutations, which is exactly what we need to do.

I think that's the main design choice I made. Lists were the obvious choice for storing permutations because we have to process every item in the data structure and lookup speed isn't important. 

I believe the time and space complexity of this algorithm are both O(n!); a 2-letter string will have 2 permutations ($2*1$), a 3-letter string will have 6 ($3*2*1$), and so on. It's admittedly slow, because it's a factorial problem, and it's naturally going to take longer for longer strings. As a precaution, I limit the allowed length of "t" to 8 characters to prevent the function from taking too much time or memory.

# Question 2

Given a string a, find the longest palindromic substring contained in a. Your function definition should look like question2(a), and return a string.



In [293]:
from numpy import argmax
from string import punctuation

def question2(a):
    if type(a) != str:
        print('Error! Not a string.')
        return None

    # Clean up the string
    a = a.lower().replace(' ', '')
    for i in punctuation:
        a = a.lower().replace(i, '')

    # Iterate over the string and save all palindromes found
    pals_found = []
    for len_check in range(2, len(a)+1):
        for ix in range(len(a)):
            pal_to_check = a[ix:ix+len_check]
            if len(pal_to_check) < 2:
                continue
            if pal_to_check == pal_to_check[::-1]:
                pals_found.append(pal_to_check)
    if len(pals_found) == 0:
        return None

    # Calculate the length of each palindrome
    pal_lens = [len(i) for i in pals_found]

    # Return the one that is longest
    return pals_found[argmax(pal_lens)]

In [294]:
print(question2('xanadu udacity'))

aduuda


In [295]:
print(question2('a man a pla!?n a .,"canal panama'))

amanaplanacanalpanama


In [296]:
print(question2(4554))

Error! Not a string.
None


This function has a few moving parts. First, I decided to clean up the string by removing spaces and punctuation from it. This wasn't specifically asked, but I think it improves the function by allowing it to find potentially longer anagrams that are within sentences.

Then I iterate over the string provided and find palindromes of increasing length. I create another list to track the lengths of each palindrome and then use NumPy's argmax function to find the index with the longest palindrome.

This function appears to have O(n^2) time and space complexity based on my understanding and testing of it. As the length of the string gets longer, it's going to take more time to a) generate palindromes, and b) search over the string. We could elect to limit the string length like I did in question 1, but even a string length of 100+ characters takes only about 20ms to process.

# Question 3

In [297]:
class Node(object):
    def __init__(self, value):
        self.value = value
        self.edges = []

class Edge(object):
    def __init__(self, value, node_from, node_to):
        self.value = value
        self.node_from = node_from
        self.node_to = node_to

class Graph(object):
    def __init__(self, nodes=[], edges=[]):
        self.nodes = nodes
        self.edges = edges

from collections import defaultdict
def question3(G):
    if type(G) != dict:
        print('Error! G is not dictionary.')
        return None

    if len(G) < 2:
        print('Error! The graph should have at least 2 nodes.')
        return None

    nodes = list(G.keys())
    edges = []

    for node in nodes:
        for connected_node in G[node]:
            try:
                a_to_b = (connected_node[1], connected_node[0], node)
                b_to_a = (connected_node[1], node, connected_node[0])
            except IndexError:
                print('Error! Was unable to connect nodes.')
                return None
            if node > connected_node[0] and a_to_b not in edges:
                edges.append(a_to_b)
            elif node < connected_node[0] and a_to_b not in edges:
                edges.append(b_to_a)

    # Sort edges and prepare for processing
    edges = sorted(edges)
    edges_processed = []
    nodes = [[i] for i in nodes]

    for edge in edges:
        from_, to_ = edge[1], edge[2]
        for node_ix in range(len(nodes)):
            if from_ in nodes[node_ix]:
                node_ix_1 = node_ix
            if to_ in nodes[node_ix]:
                node_ix_2 = node_ix

        if node_ix_1 < node_ix_2:
            nodes[node_ix_1] = set(list(nodes[node_ix_1]) + list(nodes[node_ix_2]))
            nodes.pop(node_ix_2) # remove the larger index/weight
            edges_processed.append(edge)
        if node_ix_1 > node_ix_2:
            nodes[node_ix_2] = set(list(nodes[node_ix_1]) + list(nodes[node_ix_2]))
            nodes.pop(node_ix_1)
            edges_processed.append(edge)

    # Generate the minimum spanning tree
    graph = defaultdict(list)
    for edge in edges_processed:
        from_, to_ = edge[1], edge[2]
        weight = edge[0]
        graph[from_].append((to_, weight))
        graph[to_].append((from_, weight))
    return dict(graph)

In [298]:
g = {'A': [('B', 5), ('C', 3)],
     'B': [('A', 1), ('C', 6)],
     'C': [('A', 3), ('B', 2)],
     'D': [('B', 1)]}
print(question3(g))

{'A': [('B', 1)], 'B': [('A', 1), ('D', 1), ('C', 2)], 'D': [('B', 1)], 'C': [('B', 2)]}


In [299]:
g = {'A': [('B', 5)]}
print(question3(g))

Error! The graph should have at least 2 nodes.
None


In [300]:
g = {'A': [()],
     'B': [('A', 2)],}
print(question3(g))

Error! Was unable to connect nodes.
None


I chose to solve this problem with Kruskal's algorithm. In basic terms, this algorithm sorts the edges by their ascending weights, and preferentially chooses the edges that a) add new nodes to the list of nodes visited, and b) don't complete a cycle. This process continues until the algorithm finds the graph's minimum spanning tree.

Kruskal's algorithm is known to run in O(E log E) time, where E is the number of edges in the graph. My understanding is that it has a linear space complexity of O(n), increasing with the size of the graph.

My implementation of Kruskal's algorithm is fairly straightforward. I occasionally used sets as a data structure to avoid processing nodes more than once. I also made use of the collection module's defaultdict to create a graph on-the-fly, rather than looking up whether edges were already in it. This isn't strictly necessary, but it would have been 3 more lines of code to check whether keys were in the dictionary and to add them if they weren't.

There are quite a few different errors you can make when using this function, and I think I found ways to catch all of them. The function requires a dictionary with nodes that connect to one another. If the function doesn't receive a dictionary, gets a single node, or can't connect the nodes, it outputs an error message and returns None.

# Question 4

Find the least common ancestor between two nodes on a binary search tree. The least common ancestor is the farthest node from the root that is an ancestor of both nodes. For example, the root is a common ancestor of all nodes on the tree, but if both nodes are descendents of the root's left child, then that left child might be the lowest common ancestor. You can assume that both nodes are in the tree, and the tree itself adheres to all BST properties. The function definition should look like question4(T, r, n1, n2), where T is the tree represented as a matrix, where the index of the list is equal to the integer stored in that node and a 1 represents a child node, r is a non-negative integer representing the root, and n1 and n2 are non-negative integers representing the two nodes in no particular order. For example, one test case might be 

In [301]:
def question4(T, r, n1, n2):
    if type(T) != list or type(T[0]) != list:
        print("Error! This doesn't appear to be an adjacency matrix.")
        return None
    if type(r) != int or type(n1) != int or type(n2) != int:
        print("Error! r, n1, and n2 must be integers.")
        return None
    n1_ancestors = []
    attempts = 0
    while n1 != r:
        for i in range(len(T)):
            if T[i][n1] == 1:
                n1 = i
                n1_ancestors.append(n1)
        attempts += 1
        if attempts > len(T):
            print('No common ancestor found!')
            return None
    if len(n1_ancestors) == 0:
        print("Error! N1 has no parents. (Perhaps he's Batman?)")
        return None
    attempts = 0
    while n2 != r:
        for i in range(len(T)):
            if T[i][n2] == 1:
                n2 = i
        attempts += 1
        if attempts > len(T):
            print('No common ancestor found!')
            return None
    if n2 in n1_ancestors:
        return n2
    else:
        print('No common ancestor found!')
        return None

In [302]:
print(question4([[0, 1, 0, 0, 0],
           [0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0],
           [1, 0, 0, 0, 1],
           [0, 0, 0, 0, 0]],
          r=3,
          n1=1,
          n2=4))

3


In [303]:
question4([[0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0],
           [0, 0, 0, 0, 1],
           [0, 0, 0, 0, 0]],
          r=3,
          n1=1,
          n2=4)

No common ancestor found!


In [304]:
question4("um, it's like a tree where node 0 connects to nodes 1 and 2, and node 1 connects to node 3",
          r=0,
          n1=0,
          n2=0)

Error! This doesn't appear to be an adjacency matrix.


Again, the code I used here is fairly straightforward. The main idea is to find the ancestors of both elements and identify the first ancestor of n2 that appears in the list of n1's ancestors.

This function appears to have a time complexity of O(n) because it has to traverse up the tree, and the length of the tree is going to affect how much time it takes. The space complexity is also O(n), since the most memory-intensive thing we're doing is storing a list of increasing length.

# Question 5

Find the element in a singly linked list that's m elements from the end. For example, if a linked list has 5 elements, the 3rd element from the end is the 3rd element. The function definition should look like question5(ll, m), where ll is the first node of a linked list and m is the "mth number from the end". You should copy/paste the Node class below to use as a representation of a node in the linked list. Return the value of the node at that position.

In [305]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList(object):
    def __init__(self, head=None):
        self.head = head

    def append(self, data):
        current = self.head
        if self.head:
            while current.next:
                current = current.next
            current.next = data
        else:
            self.head = data

    def position(self, position):
        count = 1
        current_pos = self.head
        if position < 1:
            return None
        while count <= position and current_pos:
            if count == position:
                return current_pos
            count += 1
            current_pos = current_pos.next
        return None

    def __len__(self):
        current = self.head
        count = 0
        while current:
            count += 1
            current = current.next # continue until next is None
        return count

def question5(ll, m):
    try:
        return ll.position(len(ll) + 1 - m).data
    except:
        print("Error! You didn't provide a valid position.")
        return None

In [307]:
ll = LinkedList()

# Append some sequential numbers
for i in range(1,11):
    ll.append(Node(i))

print(question5(ll, 1))

10


In [308]:
print(question5(ll, 2))

9


In [309]:
print(question5(ll,0))

Error! You didn't provide a valid position.
None


In [310]:
ll = LinkedList() # Try it with an empty list
print(question5(ll,1))

Error! You didn't provide a valid position.
None


I built upon Udacity's implementation of a linked list in order to make this work. There are a few methods we'll need to add to the LinkedList class:

- The `.append` method, as its name suggests, allows us to append data to the list. It looks over each node until it finds the last one (where self.next is None) and then appends the data there.

- The `.position` method is similar to indexing, but it starts counting at 1.

- The `.__len__` method returns the length of the list.

With these methods implemented, the function for question 5 is extremely simple! It just calls the .position method and returns the data "m" steps back from the end of the list. I followed the pythonic principle of asking for forgiveness rather than permission by just attempting this right away, and then returning an error if it doesn't work for whatever reason.

This function has a time and space complexity of O(n) because it has to iterate over the linked list to find the correct position.